In [1]:
# Import the required libraries

import numpy as np
import pandas as pd
import os

## Load data

In [2]:
# First, we load dataframes with 'pandas.read_csv': 

# Saber 11

df1 = pd.read_csv("saber11_2019_2022_cleaned.csv",
                  delimiter=",",
                  index_col=0)

# Scholar census

df2 = pd.read_csv("acceso_tecnologico.csv",
                  delimiter=",",
                  index_col=0)

## Merging dataframes

In [3]:
# Now, we merge the dataframes based on the keys: 1)'Código DANE de la Sede educativa' and 2) 'Año'.

# left_on=['cole_cod_dane_sede', 'ano'], right_on=['SEDE_CODIGO', 'PERIODO_ANIO'] and then drop redundant columns:

data = (pd.merge(left=df1,
                 right=df2,
                 left_on=['cole_cod_dane_sede', 'ano'],
                 right_on=['SEDE_CODIGO', 'PERIODO_ANIO'],
                 how='inner')
    .drop(columns=['SEDE_CODIGO', 'PERIODO_ANIO'], axis=1))

## Creating interaction and diff-in-diff variable

In [4]:
# Interaction between family having computer and internet:

data['HOGAR_TI'] = (data['fami_tienecomputador']
                    .mul(data['fami_tieneinternet']))

In [5]:
# Interaction between school having computer and internet:

data['SEDE_TI'] = (data['SEDETE_INTERNET']
                    .mul(data['SEDETE_EQUIPO_COMPUTO']))

In [6]:
# Interaction between year and family having computer and internet

data['2022_X_HOGAR_TI'] = (data['ano_2022']
                            .mul(data['HOGAR_TI']))

In [7]:
# Interaction between year and school having computer and internet

data['2022_X_SEDE_TI'] = (data['ano_2022']
                            .mul(data['SEDE_TI']))

In [8]:
# Interaction between family and school having computer and internet:

data['ENTORNO_TECNOLOGICO'] = (data['fami_tienecomputador']
                               .mul(data['fami_tieneinternet'])
                               .mul(data['SEDETE_INTERNET'])
                               .mul(data['SEDETE_EQUIPO_COMPUTO']))

In [9]:
# Interaction between year and family and school having computer and internet:

data['2022_X_ENTORNO_TECNOLOGICO'] = (data['ano_2022']
                                      .mul(data['ENTORNO_TECNOLOGICO']))

## Transform column names

In [10]:
# Using the rename function, we transform the column names in Capitalize:

data = data.rename(columns=str.capitalize)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 661742 entries, 0 to 661741
Data columns (total 38 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Cole_bilingue               661742 non-null  float64
 1   Cole_cod_dane_sede          661742 non-null  float64
 2   Cole_cod_depto_ubicacion    661742 non-null  float64
 3   Cole_cod_mcpio_ubicacion    661742 non-null  float64
 4   Estu_inse_individual        661742 non-null  float64
 5   Estu_nse_establecimiento    661742 non-null  float64
 6   Estu_nse_individual         661742 non-null  float64
 7   Fami_educacionmadre         661742 non-null  float64
 8   Fami_educacionpadre         661742 non-null  float64
 9   Fami_estratovivienda        661742 non-null  float64
 10  Fami_tieneautomovil         661742 non-null  float64
 11  Fami_tienecomputador        661742 non-null  float64
 12  Fami_tieneinternet          661742 non-null  float64
 13  Fami_tienelava

## Standardize test scores

The idea here is to standardize the test scores (ie. to have mean of zero and a variance of 1) by computing the test scores (global, matematicas, ingles, ciencias natutales, etc ) as $z = \frac{x - \mu}{\sigma}$, where:
- $\mu$: is the mean of the popultaion.
- $\sigma$ is the standard deviation of the population.

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [13]:
# Define the columns to standardize

names = ['Punt_ingles', 'Punt_matematicas', 'Punt_sociales_ciudadanas',
         'Punt_c_naturales', 'Punt_lectura_critica', 'Punt_global']

In [14]:
# Standarizing the tests scores

data2 = scaler.fit_transform(data.reindex(columns=names))

data2

array([[ 0.25928394, -0.37525616,  0.55040587, -0.30476013,  0.59429937,
         0.14244813],
       [ 0.5758508 ,  0.30753659,  0.7170404 ,  0.56306644,  0.59429937,
         0.61901498],
       [ 0.65499251,  1.07567844,  0.7170404 ,  0.27379092,  0.4009621 ,
         0.71829974],
       ...,
       [-0.53213319,  0.1368384 ,  1.05030947,  0.37021609,  0.88430528,
         0.57930108],
       [ 0.0218588 ,  0.90498025,  0.38377134,  0.46664127,  0.20762483,
         0.51973022],
       [-0.37384976,  0.81963115, -0.2827668 ,  0.37021609,  0.4009621 ,
         0.2814468 ]], shape=(661742, 6))

In [15]:
# Now we set the standardized variables from numpy.array to pandas.DataFrame

data2 = pd.DataFrame(data2, columns=['Punt_ingles_std', 'Punt_matematicas_std', 'Punt_sociales_ciudadanas_std',
                                     'Punt_c_naturales_std', 'Punt_lectura_critica_std', 'Punt_global_std'])

In [16]:
# Finally we join the standardized variables to the main dataframe

data = data.join(data2)

data

,Cole_bilingue,Cole_cod_dane_sede,Cole_cod_depto_ubicacion,Cole_cod_mcpio_ubicacion,Estu_inse_individual,Estu_nse_establecimiento,Estu_nse_individual,Fami_educacionmadre,Fami_educacionpadre,Fami_estratovivienda,...,2022_x_hogar_ti,2022_x_sede_ti,Entorno_tecnologico,2022_x_entorno_tecnologico,Punt_ingles_std,Punt_matematicas_std,Punt_sociales_ciudadanas_std,Punt_c_naturales_std,Punt_lectura_critica_std,Punt_global_std
0,0.0,3.257540e+11,25.0,25754.0,65.942505,3.0,4.0,11.0,19.0,2.0,...,0.0,0.0,1.0,0.0,0.259284,-0.375256,0.550406,-0.304760,0.594299,0.142448
1,0.0,1.730010e+11,73.0,73001.0,60.601929,2.0,3.0,16.0,19.0,2.0,...,0.0,0.0,1.0,0.0,0.575851,0.307537,0.717040,0.563066,0.594299,0.619015
2,0.0,3.118480e+11,11.0,11001.0,62.905838,3.0,3.0,19.0,17.0,3.0,...,0.0,0.0,1.0,0.0,0.654993,1.075678,0.717040,0.273791,0.400962,0.718300
3,0.0,1.470011e+11,47.0,47001.0,42.574894,2.0,2.0,14.0,6.0,1.0,...,0.0,0.0,0.0,0.0,-1.481834,-0.972700,-0.699353,-0.786886,-0.662393,-0.949684
4,0.0,1.050010e+11,5.0,5001.0,57.060596,2.0,3.0,11.0,11.0,3.0,...,0.0,0.0,1.0,0.0,-0.611275,-0.545954,-1.449209,-0.979736,-0.565724,-0.969541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661737,0.0,3.110010e+11,11.0,11001.0,71.410698,4.0,4.0,19.0,19.0,3.0,...,1.0,1.0,1.0,1.0,2.158685,2.014518,1.633530,1.623743,1.464317,1.929574
661738,0.0,1.850010e+11,85.0,85001.0,52.736797,2.0,3.0,14.0,8.0,2.0,...,1.0,1.0,1.0,1.0,-0.611275,-1.314096,-0.782670,-2.233264,0.110956,-1.128397
661739,0.0,1.760010e+11,76.0,76001.0,53.063957,3.0,3.0,14.0,11.0,3.0,...,1.0,1.0,1.0,1.0,-0.532133,0.136838,1.050309,0.370216,0.884305,0.579301
661740,1.0,1.660880e+11,66.0,66088.0,43.463154,2.0,2.0,6.0,6.0,3.0,...,0.0,1.0,0.0,0.0,0.021859,0.904980,0.383771,0.466641,0.207625,0.519730


## Summary statistics

In [17]:
# We calculate the summary statistics by institution area, i.e. Rural, Urbano

sum_stat1 = (data.groupby(['Cole_area_ubicacion_rural', 'Cole_area_ubicacion_urbano'])
    .agg(['count', 'mean', 'std'])
    .stack(future_stack=True)
    .T)

In [18]:
# We calculate the summary statistics by institution type, i.e. Oficial, No oficial

sum_stat2 = (data.groupby(['Cole_naturaleza_no oficial', 'Cole_naturaleza_oficial'])
    .agg(['count', 'mean', 'std'])
    .stack(future_stack=True)
    .T)

In [19]:
# We calculate the summary statistics by gender, i.e. male, female

sum_stat3 = (data.groupby(['Estu_genero_f', 'Estu_genero_m'])
    .agg(['count', 'mean', 'std'])
    .stack(future_stack=True)
    .T)

In [20]:
# We calculate the summary statistics by year:

sum_stat4 = (data.groupby(['Ano'])
    .agg(['count', 'mean', 'std'])
    .stack(future_stack=True)
    .T)

## Exporting data

In [21]:
# data.to_csv("final_principal.csv")

In [22]:
# sum_stat1.to_excel("summary_statistics_area.xlsx")

In [23]:
# sum_stat2.to_excel("summary_statistics_type.xlsx")

In [24]:
# sum_stat3.to_excel("summary_statistics_gender.xlsx")

In [25]:
# sum_stat4.to_excel("summary_statistics_year.xlsx")